## package install

```
$ pip install oandapyV20
$ pip install backtesting==0.1.7
$ pip install pyalgotrade
$ pip install backtrader
$ pip install TA-Lib
$ brew install ta-lib

$ conda install pandas
$ conda install configparser
$ conda install requests
```

In [29]:
import configparser
import datetime
import pandas as pd
from oanda import Client
config = configparser.ConfigParser()
config.read("../oanda_config.txt")
api_key = config["Practice"]["api_key"]
oanda = Client(api_key, "", "practice")

In [30]:
UTC = datetime.timezone.utc
date_from = datetime.datetime(2020, 8, 1, 0, 0, tzinfo=UTC)
date_to = datetime.datetime(2020, 10, 30, 23, 55, tzinfo=UTC)
candles = oanda.get_candles("USD_JPY", date_from, date_to, "M5")
print(candles)

                     Time     Open     High      Low    Close  Volume
0     2020-08-02 21:00:00  105.883  105.883  105.775  105.788       5
1     2020-08-02 21:15:00  105.793  105.796  105.789  105.790       6
2     2020-08-02 21:20:00  105.784  105.791  105.783  105.783       4
3     2020-08-02 21:25:00  105.779  105.784  105.779  105.784       2
4     2020-08-02 21:30:00  105.790  105.798  105.785  105.789       7
...                   ...      ...      ...      ...      ...     ...
18574 2020-10-30 20:35:00  104.680  104.686  104.675  104.682      72
18575 2020-10-30 20:40:00  104.681  104.682  104.671  104.682     104
18576 2020-10-30 20:45:00  104.683  104.686  104.676  104.682     155
18577 2020-10-30 20:50:00  104.683  104.685  104.669  104.670     153
18578 2020-10-30 20:55:00  104.669  104.678  104.597  104.620     169

[18579 rows x 6 columns]


In [31]:
from backtesting import Strategy
from backtesting.lib import crossover
import talib

def EMA_Backtesting(values, n):
    """
    Return exponential moving average of `values`, at
    each step taking into account `n` previous values.
    """
    close = pd.Series(values)
    return talib.EMA(close, timeperiod=n)

class EmaCrossStrategy(Strategy):

    # Define the two EMA lags as *class variables*
    # for later optimization
    n1 = 5
    n2 = 10

    def init(self):
        # Precompute two moving averages
        self.ema1 = self.I(EMA_Backtesting, self.data.Close, self.n1)
        self.ema2 = self.I(EMA_Backtesting, self.data.Close, self.n2)

    def next(self):
        # If ema1 crosses above ema2, buy the asset
        if crossover(self.ema1, self.ema2):
            self.position.close()
            self.buy()

        # Else, if ema1 crosses below ema2, sell it
        elif crossover(self.ema2, self.ema1):
            self.position.close()
            self.sell()

In [32]:
from backtesting import Backtest

candles_ = candles.copy()
bt_ema = Backtest(candles_, EmaCrossStrategy, cash=1, commission=0, trade_on_close=True)
bt_ema.run()
bt_ema.plot()

<ipython-input-32-55a7b069040c>:4: UserWarning: Data index is not datetime. Assuming simple periods.
  bt_ema = Backtest(candles_, EmaCrossStrategy, cash=1, commission=0, trade_on_close=True)
